In [1]:
import pandas as pd
import numpy as np
import jsonlines
import json
import os

import re
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/ujjwalaananth/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/ujjwalaananth/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:

addpred=False
addanyway=False



In [3]:

tofile=True

include_neg = True

if(include_neg):
    #dirs = ['mlee_processed_sent','CASIE_sent_neg_processed','cysecED_neg_processed','wikievent_sent_neg_processed','RAMS3_neg_processed','maven_processed_sent']
    #opdir = 'multitask5sent_neg'

    #dirs = ['mlee_processed_sent','wikievent_sent_neg_processed','RAMS_doc','maven_processed_sent']
    #dirs = ['mlee_processed_doc','wikievent_sent_neg_processed','RAMS_doc','maven_processed_sent']
    dirs = ['RAMS3_neg_processed']
    opdir = 'multitask4sent_bi'
    #opdir = 'multitask4mix_bin2'
    
# sent
else:
    #dirs = ['CASIE_sent_processed','cysecED_processed','wikievent_sent_processed','RAMS3_processed',]
    dirs = ['CASIE_sent_processed','cysecED_processed','wikievent_sent_processed','RAMS3_processed','maven_processed_sent']
    #opdir = 'multitask4sent'
    opdir = 'multitask5sent'

# sent, with negative

# GROUP CYBERSEC AND GEN DATASETS
domains={
    #'cybersec':dirs[1:3],
    'gen':['wikievent_sent_neg_processed','RAMS_doc'],
    'biomed2k':['mlee_processed_sent']
}

impcols = ['text', 'src_label', 'event_triggers', 'event_types']



In [4]:
st = False
for d in dirs:
    trfile = d+'/train.csv'
    tefile = d+'/test.csv'
    
    df1 = pd.read_csv(trfile)
    df2 = pd.read_csv(tefile)
    
    df1 = df1.loc[:,impcols]
    df2 = df2.loc[:,impcols]
    
    if('genia' in d):
        df1 = df1.iloc[:2000,:]
    
    print(d)
    print(df1.head())
    print(df1.shape, ' df')
    
    df1['dataset'] = [d]*len(df1)
    df2['dataset'] = [d]*len(df2)
    
    
    
    
    if(not st):
        st=True
        dfn1 = df1
        dfn2 = df2
    else:
        dfn1 = pd.concat([df1,dfn1],axis=0,ignore_index=True)
        dfn2 = pd.concat([df2,dfn2],axis=0,ignore_index=True)
        dfn1 = dfn1.sample(frac=1).reset_index(drop=True)
        #dfn2 = dfn2.sample(frac=1).reset_index(drop=True)
    print(dfn1.shape,dfn2.shape)
    
    if(include_neg):
        dfn1 = dfn1.fillna('NONE')
        dfn2 = dfn2.fillna('NONE')
    else:
        dfn1 = dfn1.dropna(subset=['src_label'])
        dfn2 = dfn2.dropna(subset=['src_label'])
    
    
    print(dfn1.shape)


RAMS3_neg_processed
                                                text           src_label  \
0  Transportation officials are urging carpool an...                 NaN   
1                                    ( Paul Duggan )                 NaN   
2  -- A Baltimore prosecutor accused a police det...  homicide->life.die   
3  The heated exchange came in the chaotic sixth ...                 NaN   
4                     ( Derek Hawkins and Lynh Bui )                 NaN   

  event_triggers event_types  
0           NONE        NONE  
1           NONE        NONE  
2       homicide    life.die  
3           NONE        NONE  
4           NONE        NONE  
(35535, 4)  df
(35535, 5) (4245, 5)
(35535, 5)


In [5]:
def unique_sents(df):
    for sent in df.text.unique():
        ds = df[df.text==sent]
        if(len(ds)==1):
            continue
        ids = ds.index
        tgs = ds.src_label.unique()
        if('NONE' in tgs and len(tgs)>1):
            tgs = [t for t in tgs if 'NONE' not in t]
        df.loc[ids[0],'src_label'] = ' | '.join(tgs)
        #print(df.loc[ids[0],'src_label'])
        
    print(df.shape)
    df = df.drop_duplicates(subset=['text'])
    print(df.shape,' after dropping dups')
    return df

In [6]:
## PROCESSING WIKIEVENT

for df in [dfn1,dfn2]:
    
    for i,r in df.iterrows():
        if('NONE' in r['src_label'] or 'wiki' not in r['dataset']):
            continue
            
        et = [p.strip() for p in r['event_types'].split('|')]
        et = ['.'.join(p.split('.')[:-1]) for p in et]
        r['event_types'] = ' | '.join(et)
        
        et = [p.strip() for p in r['src_label'].split('|')]
        et = ['.'.join(p.split('.')[:-1]) for p in et]
        r['src_label'] = ' | '.join(et)
        

In [7]:
dfn1 = unique_sents(dfn1)
dfn2 = unique_sents(dfn2)

(35535, 5)
(27751, 5)  after dropping dups
(4245, 5)
(3456, 5)  after dropping dups


In [18]:
for dataset in dfn1.dataset.unique():
    print('dataset ',dataset)
    
    print(dfn1[dfn1.dataset==dataset].shape[0])
    print(dfn1[dfn1.dataset==dataset].text.nunique())
    
    print(dfn2[dfn2.dataset==dataset].shape[0])
    print(dfn2[dfn2.dataset==dataset].text.nunique())
    

dataset  RAMS3_neg_processed
27751
27751
3456
3456


In [19]:
print(dfn1.head())
print('=======')
print(dfn2.head())

                                                text           src_label  \
0  Transportation officials are urging carpool an...                NONE   
1                                    ( Paul Duggan )                NONE   
2  -- A Baltimore prosecutor accused a police det...  homicide->life.die   
3  The heated exchange came in the chaotic sixth ...                NONE   
4                     ( Derek Hawkins and Lynh Bui )                NONE   

  event_triggers event_types              dataset  
0           NONE        NONE  RAMS3_neg_processed  
1           NONE        NONE  RAMS3_neg_processed  
2       homicide    life.die  RAMS3_neg_processed  
3           NONE        NONE  RAMS3_neg_processed  
4           NONE        NONE  RAMS3_neg_processed  
                                                text  \
0                         We are ashamed of them . "   
1  However , Mutko stopped short of admitting the...   
2  " We are very sorry that athletes who tried to...   
3  We a

In [20]:
dfn2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3456 entries, 0 to 4244
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            3456 non-null   object
 1   src_label       3456 non-null   object
 2   event_triggers  3456 non-null   object
 3   event_types     3456 non-null   object
 4   dataset         3456 non-null   object
dtypes: object(5)
memory usage: 162.0+ KB


In [21]:
dctp={}
dctg={}
predevex=open('results/'+'multitask2EV_prin_1024_20_2bs_2DAYS'+'/generated_predictions.txt').readlines()

len(predevex)

2913

In [22]:
predevex=[p[2:-2] for p in predevex]
predevex[0:5]

['contact', 'deceive', 'deceive->contact', 'conflict', 'slaughtered']

In [23]:
dfl=[]

In [24]:
def getdef(wordl):
    
    
    defs=''
    
    for word in wordl.split(' | '):
        if(len(word.split())>1):
            word = max(word.split(), key=len)
            
        if(len(word.split('.'))>1):
            word = word.split('.')[1] if len(wordnet.synsets(word.split('.')[1]))>0 else word.split('.')[0]
        if(len(wordnet.synsets(word))>0):
            defs+= wordnet.synsets(word)[0].definition()+' | '
            if(len(wordnet.synsets(word))>1):
                defs+= wordnet.synsets(word)[1].definition()+' | '
        else:
            defs+= ''

    return defs

def cleanup(word,dataset,tpe=False):
    ## for casie store second half
    ## do substitution
            
    return re.sub("-", " ",word)


def clean(w):
      
    w = w.strip()
    changed=False
    if(w[-1]=='|'):
        #print(w)
        changed=True
        w = w[:-1].strip()
        #print(w)
        
    w = ' | '.join(list(set([x.strip() for x in w.strip().split('|')])))
    return w,changed

def clean_src_label(opdf,relcol='src_label'):
    z=[]
    for i,r in opdf.iterrows():
        srcs = [p.strip() for p in r[relcol].split('|')]
        srcs = [s for s in srcs if 'NONE' not in s]
        if(len(srcs)==0):
            srcs = ['NONE']
        z.append(' | '.join(srcs))
    opdf[relcol] = z
    return opdf

In [25]:
#def process(df,opind)
dfl=[]

z=0

def process(df,opind,remove,testfile=False,include_neg=False):
    tasks=['EV-CLF','EV-ID','EV-EX']#,'ARG-ID','ARG-EX']
    rel_cols=['event_types','event_triggers','src_label']#,'args','src_label2']
    
    qns = ['What types of events are occuring in this text?',
          'Which words or phrases denote events occurring in this text?',
        'What are the events occurring in this text?']
    
    instr=['This input text gives information about specific types of ongoing events. The output should be the types of events occurring in the text. If there are no events, print NONE',
           'You are given a text as input. The text gives information about ongoing events. An event nugget is a word or phrase that most clearly expresses the event occurrence. Your task is to identify the words or phrases that are event nuggets for events in the text, where event type is not given. If there are no events, print NONE',
           'The text given as input discusses ongoing events. An event nugget is a word or phrase that most clearly expresses the event occurrence. Generate output in the format [event nugget->event type] for all events in the text. If there are no events, print NONE'
           #'An event argument is an entity that is directly or indirectly involved in the event. It acts on or is acted on by the nugget. From the text given as input, extract the relevant arguments for the event in this text, where event type and nugget are not given',
           #'The following text talks about events and related entities. An event argument is an actor that is directly or indirectly involved in the event. It acts on or is acted on by the nugget. The role it plays depends on the event type. Your task is to generate as output, the relevant arguments for the event in this text and classify their roles. The output format is [argument]->[role]'
          ]
    
    example_c = ['INPUT: Our best analysis is that email addresses and allrecipes.com passwords were intercepted during account registration or login by our members.</s>OUTPUT: attack.</s>EXPLANATION: In this event, a malicious entity is attempting to cause harm by attacking the allrecipes.com. This is indicated by the phrase "were intercepted" which shows active hostile actions.',
                 'INPUT: W-2 records for all companies and all employees were compromised.</s>OUTPUT: were compromised</s>EXPLANATION: The event occurrence is described by the phrase "were compromised" in the given text.',
                 'INPUT: It has also been suggested that the criminals who stole the data feared detection if they sold it online and have kept it to themselves to avoid capture.</s>OUTPUT: stole->attack</s>EXPLANATION: The event is an attack, as indicated by the word "steal" which functions as the event nugget.'
                ]
    
    example_b = ['INPUT: Left ventricular weight, body weight, and their ratio were not significantly altered by alinidine treatment.</s>OUTPUT: planned_process | regulation</s>EXPLANATION: The input contains multiple events of planned_process and regulation type',
               'INPUT: Left ventricular weight, body weight, and their ratio were not significantly altered by alinidine treatment.</s>OUTPUT: treatment | altered</s>EXPLANATION: the words "treatment" and "altered" are salient words describing important events.',
               'INPUT: Left ventricular weight, body weight, and their ratio were not significantly altered by alinidine treatment.</s>OUTPUT: treatment->planned_process | altered->regulation</s>EXPLANATION:  The word "treatment" in the input denotes a planned process, while the "altered" indicates the sentence talks about regulation.'
               #'INPUT: Last spring, hackers got into the system at the ministry, which was then headed by now-Prime Minister Paolo Gentiloni, and the attacks carried on for more than four months but did not  gain access to classified information, the paper said. OUTPUT: system | hackers | the ministry |  EXPLANATION: The event here, classified as attack and triggered by the phrase "got into", directly involves the following entities: system, which is affected, the ministry, which is also affected, hackers, who carry it out.',
               #'INPUT: As Action Fraud warns, confidence tricksters are phoning up schools and colleges pretending to be from the Department of Education. OUTPUT: Department of Education->Trusted-Entity | colleges->Victim | schools->Victim | tricksters->Attacker |  EXPLANATION: In the cybersecurity event described, the event is done by the tricksters in the role of Attacker. It occurs through the Department of Education, which functions as a Trusted-Entity. The event adversely affects two entities: colleges and schools, which thus act as Victim entities.'
              ]
    example_r = ['INPUT:  The leaflets carried several messages to the citizens attempting to reassure them that the advancing army " would not target civilians , " but warned them to avoid the known locations of Isis militants . The military operation is the most complex carried out in Iraq since US forces withdrew from the country in 2011 . Last week , the UN said it was bracing itself for the world \'s biggest and most complex humanitarian effort following the battle , which it expects will displace up to one million people and see civilians used as human shields.</s>OUTPUT: conflict.</s>EXPLANATION: The event triggered by "battle" refers to an event of the type "conflict" which refers to a serious disagreement between two or more entities',
               'INPUT: The information minister alleged that oil smuggled into Turkey was bought by the Turkish president\'s son , who owns an oil company . Mr al - Zoubi said in an interview , All of the oil was delivered to a company that belongs to the son of Recep [ Tayyip ] Erdogan . This is why Turkey became anxious when Russia began delivering airstrikes against the IS infrastructure and destroyed more than 500 trucks with oil already.</s>OUTPUT: smuggled</s>EXPLANATION: The event describes goods being moved. The exact nugget from the text that describes this event is "smuggled"',
               'INPUT: The Organization for Security and Cooperation In Europe \'s ( OSCE ) Office for Democratic Institutions and Human Rights and the OSCE High Commissioner on National Minorities issued a report in September saying that since Russia \'s land grab , fundamental freedoms had " deteriorated radically " for many in Crimea , especially for pro - Ukrainian activists , journalists , and the Crimean Tatar community.</s>OUTPUT: land grab->transaction.exchangebuysell</s>EXPLANATION: In this text, the event being discussed is the "land grab", which functions as the event nugget. The type of event it describes is a transaction, in which ownership of entities is transferred.'
               #'INPUT: Russian President Vladimir Putin implicated in fatal poisoning of former KGB operative. That cautious stance is likely to disappoint Litvinenko \'s wife , who called Thursday for Britain to expel Russian intelligence officials and enact new sanctions in response to the killing of her husband . OUTPUT: Russian President Vladimir Putin | former KGB operative | poisoning  OUTPUT: In the described event, the Russian President Vladimir Putin does the event, and the event occurs through the entity poisoning. The poisoning acts on a former KGB operative, who is the victim of this poisoning attack carried out by Putin.',
               #'INPUT: Whatever the technicalities , the big picture is this : unless the level of Russian airstrikes dramatically decreases , this ceasefire will not hold because the moderate armed opposition can not lay down their weapons and will not lay down their weapons while they are being annihilated from the air . OUTPUT:  | Russian->attacker | the moderate armed opposition->target | airstrikes->instrument | the air->place  EXPLANATION: The entities involved in this event are the Russian, who are the attacker. The airstrikes are the instrument through which the attack is carried out, on the target which is the moderate armed opposition. The place argument for the event is the air.']
              ]
    prompts=['This passage talks about an event. Extract the type of event being talked about.',
             'An event nugget is a word or phrase that most clearly expresses the event occurrence. Which word or phrase is the event nugget in this text?',
             'This passage talks about an event. An event nugget is a word or phrase that most clearly expresses the event occurrence. Which word or phrase is the event nugget in this text and what type of event does it refer to?'
             #'An event argument is an agent or actor that is directly or indirectly involved in the event. It acts on or is acted on by the nugget. Which are the relevant arguments for the event in this passage?',
             #'An event argument is an agent or actor that is directly or indirectly involved in the event. It acts on or is acted on by the nugget in a particular role. Which are the relevant arguments and what roles do they play for the event in this passage?'
            ]
    
    #"You are given a context as in input. This passage gives information about ongoing event. Your job is to generate a event type as an output, where event trigger is not given."

    #"Prompt: " + prompt + ', ' + 'context: ' + hdsk + ', ' + 'output: ?'
    
    #"Definition: " + + "example_1: [input: , output: , explanation: ]" + "Input: , output: ?"  
    
    text=[]
    texto=[]
    textevex=[]
    textevp=[]
    subtask=[]
    dataset=[]
    src_label=[]
    text_p=[]
    text_in=[]
    text_in_only=[]
    
    context=[]    
    text_golddef=[]
    instruction=[]
    examples=[]
    
    print(len(df),' is len')

    for ind,row in df.iterrows():
        
        #if(row['text'] in example_b[0]):
        #    continue
        
        for i,t in enumerate(tasks):
            
            #if(t=='EV-EX'):
            #    dctg[row['text']]=row['src_label']
                #print(predevex[ind][2:-2])
                    
            if(t=='EV-CLF'): # first task
                dctg[row['text']]={}
                ## for casie store second half
                dctg[row['text']]['etyg']=cleanup(row[rel_cols[i]],row['dataset'],tpe=True)
                
            if(t=='EV-ID'):
                dctg[row['text']]['evg']=cleanup(row[rel_cols[i]],row['dataset'],tpe=False)
                
            subtask.append(t)
            
            '''
            if(t in domains['gen']):
                example_rel = example_r
            else:
                example_rel = example_b
            '''
            lab = row[rel_cols[i]]
            
            if((include_neg and len(lab)==0) or lab=='NONE->NONE'):
                lab = 'NONE'
                
            lab,changed = clean(lab)
            
            src_label.append(lab)
            dataset.append(row['dataset'])
            texto.append(row['text'])
            text.append(subtask[-1]+'</s>'+dataset[-1]+'</s>'+row['text'])
            #text_p.append(' PROMPT: '+prompts[i]+' INPUT: '+row['text']+' OUTPUT: ?')
            text_in.append(instr[i]+' EXAMPLE_1: ['+example_b[i]+'] EXAMPLE_2: ['+example_r[i]+'] INPUT: '+row['text']+' OUTPUT: ?' )
            #text_in.append('context: '+row['text']+'</s>question: '+qns[i] )
            #text_in.append(instr[i]+'</s>EXAMPLE_1: ['+example_r[i]+'</s>EXAMPLE_2: ['+example_b[i]+'</s>EXAMPLE_3: ['+example_c[i]+']</s>INPUT: '+row['text']+'</s>OUTPUT: ?' )
            
            context.append(instr[i]+' EXAMPLE_1: ['+example_b[i]+'] EXAMPLE_2: ['+example_r[i]+']')
            instruction.append(instr[i])
            examples.append('<example1>'+example_b[i]+'</example1><example2>'+example_r[i]+'</example2>')
            #text_golddef.append
            '''
            if('ARG-' in t):
                if(testfile):
                    ### currently using gold ev-ex labels.
                    textevex.append('EVENT: '+dctg[row['text']]+' TEXT: '+text_in[-1])
                    textevp.append('EVENT: '+dctg[row['text']].split('->')[0]+' TEXT: '+text_in[-1])
                    ### currently using predicted ev-ex labels 
                    #textevex.append(dctp[row['text']]+'<s>'+text[-1])
                else:
                    textevex.append('EVENT: '+dctg[row['text']]+' TEXT: '+text_in[-1])
                    textevp.append('EVENT: '+dctg[row['text']].split('->')[0]+' TEXT: '+text_in[-1])
            else:
                textevex.append('')
                textevp.append('')
            '''    
            
            

    
    #opdf = pd.DataFrame(data={'prompt':text_in_only,'examples':examples,'text':text,'question':texto,'text_pr':text_p,'text_in':text_in,'src_label':src_label,'completion':src_label,'subtask':subtask,'dataset':dataset})
    opdf = pd.DataFrame(data={'text':text,'question':texto,'text_in':text_in,'src_label':src_label,'subtask':subtask,'dataset':dataset})
    
    '''
    opdf['text_evg']=['']*len(opdf)
    opdf['text_evp']=['']*len(opdf)
    opdf['text_evtyg']=['']*len(opdf)
    opdf['text_evtyp']=['']*len(opdf)
    
    for ip,r in opdf.iterrows():
        tg=dctg[r['texto']]['evg']
        tyg=dctg[r['texto']]['etyg']
        
        r['text_evg'] = 'EVENT: '+tg+' DEFINITION: '+getdef(tg)+' TASK: '+r['text_in']
        r['text_evp'] = 'EVENT: '+tg+' DEFINITION: '+getdef(tg)+' TASK: '+r['text_in']
        r['text_evtyg'] = 'EVENT: '+tyg+' DEFINITION: '+getdef(tyg)+' TASK: '+r['text_in']
        r['text_evtyp'] = 'EVENT: '+tyg+' DEFINITION: '+getdef(tyg)+' TASK: '+r['text_in']
    
    '''
    if(not testfile):
        opdf = opdf.sample(frac=1).reset_index(drop=True)
    else:
        print('orilen ',len(opdf))
        opdf = opdf.drop(index=remove).reset_index(drop=True)
        print('newlen ',len(opdf))
        
    opdf = clean_src_label(opdf,relcol='src_label')
    dfl.append(opdf)
    
    if(tofile):
        opdf.to_csv(opfiles_csv[opind],index=False)
        opdf.to_json(opfiles_jsonl[opind],orient='records',lines=True,force_ascii=True)
        print('filed')
    print(opdf.head())
    print('--------------------')

In [26]:
#remove=[4302,3029,11,120,2693]
remove=[]
### fix this!!!

In [27]:

opfiles_csv = [opdir+'/train.csv',opdir+'/test.csv']
opfiles_jsonl = [opdir+'/train.jsonl',opdir+'/test.jsonl']
testfile=[False,True]
for i,df in enumerate([dfn1,dfn2]):
    process(df,i,remove,testfile[i],include_neg=include_neg)
    

27751  is len
filed
                                                text  \
0  EV-CLF</s>RAMS3_neg_processed</s>It ’s no surp...   
1  EV-EX</s>RAMS3_neg_processed</s>The three days...   
2  EV-ID</s>RAMS3_neg_processed</s>Over the past ...   
3  EV-ID</s>RAMS3_neg_processed</s>View of the ho...   
4  EV-CLF</s>RAMS3_neg_processed</s>The moderator...   

                                            question  \
0  It ’s no surprise that an organization like IS...   
1  The three days of relentless bombing meant man...   
2  Over the past two years , ISIS has publicly ex...   
3  View of the hole from the “ storage space ” wh...   
4  The moderator , Fox News ’ Chris Wallace , ask...   

                                             text_in             src_label  \
0  This input text gives information about specif...                  NONE   
1  The text given as input discusses ongoing even...  wounded->life.injure   
2  You are given a text as input. The text gives ...              execut

In [28]:
#dctp

In [29]:
domains

{'gen': ['wikievent_sent_neg_processed', 'RAMS_doc'],
 'biomed2k': ['mlee_processed_sent']}

In [30]:
#a=pd.read_csv(opdir+'/train.csv')
#b=pd.read_csv(opdir+'/test.csv')
a=dfl[0]
b=dfl[1]
a.head()

,text,question,text_in,src_label,subtask,dataset
0,EV-CLF</s>RAMS3_neg_processed</s>It ’s no surp...,It ’s no surprise that an organization like IS...,This input text gives information about specif...,NONE,EV-CLF,RAMS3_neg_processed
1,EV-EX</s>RAMS3_neg_processed</s>The three days...,The three days of relentless bombing meant man...,The text given as input discusses ongoing even...,wounded->life.injure,EV-EX,RAMS3_neg_processed
2,EV-ID</s>RAMS3_neg_processed</s>Over the past ...,"Over the past two years , ISIS has publicly ex...",You are given a text as input. The text gives ...,executed,EV-ID,RAMS3_neg_processed
3,EV-ID</s>RAMS3_neg_processed</s>View of the ho...,View of the hole from the “ storage space ” wh...,You are given a text as input. The text gives ...,NONE,EV-ID,RAMS3_neg_processed
4,EV-CLF</s>RAMS3_neg_processed</s>The moderator...,"The moderator , Fox News ’ Chris Wallace , ask...",This input text gives information about specif...,NONE,EV-CLF,RAMS3_neg_processed


In [31]:
b.head()

,text,question,text_in,src_label,subtask,dataset
0,EV-CLF</s>RAMS3_neg_processed</s>We are ashame...,"We are ashamed of them . """,This input text gives information about specif...,NONE,EV-CLF,RAMS3_neg_processed
1,EV-ID</s>RAMS3_neg_processed</s>We are ashamed...,"We are ashamed of them . """,You are given a text as input. The text gives ...,NONE,EV-ID,RAMS3_neg_processed
2,EV-EX</s>RAMS3_neg_processed</s>We are ashamed...,"We are ashamed of them . """,The text given as input discusses ongoing even...,NONE,EV-EX,RAMS3_neg_processed
3,"EV-CLF</s>RAMS3_neg_processed</s>However , Mut...","However , Mutko stopped short of admitting the...",This input text gives information about specif...,NONE,EV-CLF,RAMS3_neg_processed
4,"EV-ID</s>RAMS3_neg_processed</s>However , Mutk...","However , Mutko stopped short of admitting the...",You are given a text as input. The text gives ...,NONE,EV-ID,RAMS3_neg_processed


In [32]:
#b['text_evg'][0]

In [33]:
z=[]
for i,row in a.iterrows():
    z.append(len(row['text_in'].split()))
    
pd.Series(z).describe(percentiles=[.25,.5,.75,.95])

count    83253.000000
mean       217.153880
std         17.766421
min        188.000000
25%        206.000000
50%        215.000000
75%        225.000000
95%        246.000000
max        578.000000
dtype: float64

In [34]:
pd.Series([p for p in z if p>4100]).describe()

<ipython-input-34-09c245ff25f2>:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series([p for p in z if p>4100]).describe()


count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
dtype: float64

In [35]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83253 entries, 0 to 83252
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       83253 non-null  object
 1   question   83253 non-null  object
 2   text_in    83253 non-null  object
 3   src_label  83253 non-null  object
 4   subtask    83253 non-null  object
 5   dataset    83253 non-null  object
dtypes: object(6)
memory usage: 3.8+ MB


In [36]:
#len(b[b.subtask.str.contains('ARG')])

In [37]:
'''
op=['train','test']

for i,df in enumerate([a,b]):
    df_ev = df[df['subtask'].str.contains('EV-')]
    df_arg = df[df['subtask'].str.contains('ARG-')]
    dfl.append(df_ev)
    dfl.append(df_arg)
    
    if(addpred and i==1 and addanyway):
        for ix,r in df_ev.iterrows():
            #print(predevex[ix],r['subtask'],r['src_label'])
            if(r['subtask']=='EV-CLF'): ## first task
                if(r['texto'] not in dctp.keys()):
                    dctp[r['texto']]={}
                dctp[r['texto']]['etyp'] = cleanup(predevex[ix],r['dataset'],tpe=True)
            if(r['subtask']=='EV-ID'):
                if(r['texto'] not in dctp.keys()):
                    dctp[r['texto']]={}
                dctp[r['texto']]['evp'] = cleanup(predevex[ix],r['dataset'],tpe=False)
            if(r['subtask']=='EV-EX'):
                if(r['texto'] not in dctp.keys()):
                    dctp[r['texto']]={}
                if('evp' not in dctp[r['texto']].keys()):
                    dctp[r['texto']]['evp'] = cleanup(predevex[ix].split('->')[0],r['dataset'],tpe=False)
                if('evtyp' not in dctp[r['texto']].keys()):    
                    dctp[r['texto']]['evtyp'] = cleanup(predevex[ix].split('->')[-1],r['dataset'],tpe=True)
            
    
        for ix,r in df_ev.iterrows():
            
                
            tp = dctp[r['texto']]['evp']
            typ = dctp[r['texto']]['evtyp']
            r['text_evp'] = 'EVENT: '+tp+' DEFINITION: '+getdef(tp)+' TASK: '+r['text_in']
            r['text_evtyp'] = 'EVENT: '+typ+' DEFINITION: '+getdef(typ)+' TASK: '+r['text_in']
    
    
            
    
    if(tofile):
        df_ev.to_csv(opdir+'/'+op[i]+'ev.csv',index=False)
        df_ev.to_json(opdir+'/'+op[i]+'ev.jsonl',orient='records',lines=True,force_ascii=True)
        print(df_ev.head())
        print('-------------------------')
        df_arg.to_csv(opdir+'/'+op[i]+'arg.csv',index=False)
        df_arg.to_json(opdir+'/'+op[i]+'arg.jsonl',orient='records',lines=True,force_ascii=True)
        print(df_arg.head())
        print('filed')

'''

"\nop=['train','test']\n\nfor i,df in enumerate([a,b]):\n    df_ev = df[df['subtask'].str.contains('EV-')]\n    df_arg = df[df['subtask'].str.contains('ARG-')]\n    dfl.append(df_ev)\n    dfl.append(df_arg)\n    \n    if(addpred and i==1 and addanyway):\n        for ix,r in df_ev.iterrows():\n            #print(predevex[ix],r['subtask'],r['src_label'])\n            if(r['subtask']=='EV-CLF'): ## first task\n                if(r['texto'] not in dctp.keys()):\n                    dctp[r['texto']]={}\n                dctp[r['texto']]['etyp'] = cleanup(predevex[ix],r['dataset'],tpe=True)\n            if(r['subtask']=='EV-ID'):\n                if(r['texto'] not in dctp.keys()):\n                    dctp[r['texto']]={}\n                dctp[r['texto']]['evp'] = cleanup(predevex[ix],r['dataset'],tpe=False)\n            if(r['subtask']=='EV-EX'):\n                if(r['texto'] not in dctp.keys()):\n                    dctp[r['texto']]={}\n                if('evp' not in dctp[r['texto']].keys(

In [38]:
'''df_ev.head()'''

'df_ev.head()'

In [39]:
op=['train','test']

dss = ['EV-ID','EV-CLF','EV-EX']

for dat in dss:
    for i,df in enumerate([a,b]):
        df_c = df[df['subtask'].str.contains(dat)]
        
        dfl.append(df_c)
        
        if(tofile):

            df_c.to_csv(opdir+'/'+op[i]+dat.replace('-','').lower()+'.csv',index=False)
            df_c.to_json(opdir+'/'+op[i]+dat.replace('-','').lower()+'.jsonl',orient='records',lines=True,force_ascii=True)

            print('filed')

filed
filed
filed
filed
filed
filed


In [40]:
op=['train','test']

dss = ['RAMS','wikievent','maven','ml']

for dat in dss:
    for i,df in enumerate([a,b]):
        df_c = df[df['dataset'].str.contains(dat)]
        
        dfl.append(df_c)
        
        if(tofile):

            df_c.to_csv(opdir+'/'+op[i]+dat.lower()+'.csv',index=False)
            df_c.to_json(opdir+'/'+op[i]+dat.lower()+'.jsonl',orient='records',lines=True,force_ascii=True)
            
            print(len(df_c))
            print('filed')

83253
filed
10368
filed
0
filed
0
filed
0
filed
0
filed
0
filed
0
filed


In [41]:
df_c.head()

,text,question,text_in,src_label,subtask,dataset


In [42]:
b.iloc[0]

text         EV-CLF</s>RAMS3_neg_processed</s>We are ashame...
question                            We are ashamed of them . "
text_in      This input text gives information about specif...
src_label                                                 NONE
subtask                                                 EV-CLF
dataset                                    RAMS3_neg_processed
Name: 0, dtype: object

In [43]:
#dft=dfl[1]
#dft.iloc[2693]

In [44]:
dftrain=dfl[0]
dftest=dfl[1]

In [45]:
for dat in dss:

    for t in ['EV-EX','EV-CLF','EV-ID']:#,'ARG-EX','ARG-ID']:
        tk=t.lower().replace('-','')
        pt = dftest[dftest['subtask']==t]
        pt = pt[pt.dataset.str.contains(dat)]
        if(tofile):
            pt.to_csv(opdir+'/test'+dat.lower()+tk+'.csv',index=False)
            pt.to_json(opdir+'/test'+dat.lower()+tk+'.jsonl',orient='records',lines=True,force_ascii=True)
            print('filed')

        pt = dftrain[dftrain['subtask']==t]
        pt = pt[pt.dataset.str.contains(dat)]
        if(tofile):
            pt.to_csv(opdir+'/train'+dat.lower()+tk+'.csv',index=False)
            pt.to_json(opdir+'/train'+dat.lower()+tk+'.jsonl',orient='records',lines=True,force_ascii=True)
            print('filed')

filed
filed
filed
filed
filed
filed
filed
filed
filed
filed
filed
filed
filed
filed
filed
filed
filed
filed
filed
filed
filed
filed
filed
filed


In [46]:
'''tasks=['EV-CLF','EV-ID','EV-EX','ARG-ID','ARG-EX']
for d in ['CASIE','RAMS']
    for t in tasks:'''

"tasks=['EV-CLF','EV-ID','EV-EX','ARG-ID','ARG-EX']\nfor d in ['CASIE','RAMS']\n    for t in tasks:"

In [47]:

for dm in domains.keys():
    pt = dftrain[dftrain.dataset.isin(domains[dm])]
    pt.to_csv(opdir+'/train'+dm.lower()+'.csv',index=False)
    
    pt = dftest[dftest.dataset.isin(domains[dm])]
    pt.to_csv(opdir+'/test'+dm.lower()+'.csv',index=False)
    #dftest

In [48]:
domains

{'gen': ['wikievent_sent_neg_processed', 'RAMS_doc'],
 'biomed2k': ['mlee_processed_sent']}

In [49]:
for x in ['train.csv','test.csv']:
    df = pd.read_csv(opdir+'/'+x)
    print(df.text_in.str.split().str.len().describe())
    print('\n\n')
    #print(df.prompt.str.split().str.len().describe())
    #print('\n\n')
    print(df.text.str.split().str.len().describe())
    print('---------------------------')

count    83253.000000
mean       217.153880
std         17.766421
min        188.000000
25%        206.000000
50%        215.000000
75%        225.000000
max        578.000000
Name: text_in, dtype: float64



count    83253.000000
mean        27.820547
std         17.647194
min          1.000000
25%         16.000000
50%         25.000000
75%         36.000000
max        386.000000
Name: text, dtype: float64
---------------------------
count    10368.000000
mean       216.593750
std         16.991997
min        188.000000
25%        205.000000
50%        214.000000
75%        225.000000
max        399.000000
Name: text_in, dtype: float64



count    10368.000000
mean        27.260417
std         16.867286
min          1.000000
25%         16.000000
50%         25.000000
75%         35.000000
max        207.000000
Name: text, dtype: float64
---------------------------


In [50]:
for x in ['train.csv','test.csv']:
    df = pd.read_csv(opdir+'/'+x)
    print(df[df.text.str.split().str.len()>512]['dataset'].value_counts())
    print('\n\n')
    #print(df[df.prompt.str.split().str.len()>512]['dataset'].value_counts())
    #print('\n\n')
    print(df[df.text_in.str.split().str.len()>512]['dataset'].value_counts())
    print('-----------------------')
#print(df[df.text_in.str.len()>1024]['src_label'].iloc[2])

Series([], Name: dataset, dtype: int64)



RAMS3_neg_processed    6
Name: dataset, dtype: int64
-----------------------
Series([], Name: dataset, dtype: int64)



Series([], Name: dataset, dtype: int64)
-----------------------


In [51]:
for x in ['train.csv','test.csv']:
    df = pd.read_csv(opdir+'/'+x)
    print(df[df.text.str.split().str.len()>1024]['dataset'].value_counts())
    print('\n\n')
    #print(df[df.prompt.str.split().str.len()>1024]['dataset'].value_counts())
    #print('\n\n')
    print(df[df.text_in.str.split().str.len()>1024]['dataset'].value_counts())
    print('-----------------------')
#print(df[df.text_in.str.len()>1024]['src_label'].iloc[2])

Series([], Name: dataset, dtype: int64)



Series([], Name: dataset, dtype: int64)
-----------------------
Series([], Name: dataset, dtype: int64)



Series([], Name: dataset, dtype: int64)
-----------------------


In [52]:
for x in ['train.csv','test.csv']:
    df = pd.read_csv(opdir+'/'+x)
    print(df[df.text.str.split().str.len()>2048]['dataset'].value_counts())
    print('\n\n')
    #print(df[df.prompt.str.split().str.len()>2048]['dataset'].value_counts())
    #print('\n\n')
    print(df[df.text_in.str.split().str.len()>2048]['dataset'].value_counts())
    print('-------------------')
#print(df[df.text_in.str.len()>1024]['src_label'].iloc[2])

Series([], Name: dataset, dtype: int64)



Series([], Name: dataset, dtype: int64)
-------------------
Series([], Name: dataset, dtype: int64)



Series([], Name: dataset, dtype: int64)
-------------------


In [53]:
for x in ['train.csv','test.csv']:
    df = pd.read_csv(opdir+'/'+x)
    print(df.dataset.value_counts()/3)

RAMS3_neg_processed    27751.0
Name: dataset, dtype: float64
RAMS3_neg_processed    3456.0
Name: dataset, dtype: float64


In [54]:
'''for x in ['train.csv','test.csv']:
    df = pd.read_csv(opdir+'/'+x)
    print(df[df.dataset.str.contains('genia') & df.subtask.str.contains('EX')]['src_label'].value_counts())'''

"for x in ['train.csv','test.csv']:\n    df = pd.read_csv(opdir+'/'+x)\n    print(df[df.dataset.str.contains('genia') & df.subtask.str.contains('EX')]['src_label'].value_counts())"